In [7]:
# sys path
import sys
sys.path.append('../../')

In [8]:
from holisticai.utils.search.synthetic_data import generate_rankings

M = 10000
top_n = 10
p = 0.2
rankings = generate_rankings(M, top_n, p)
rankings['gender'] = rankings['gender']=='Female'

In [9]:
rankings.head(10)

,query_id,doc_id,score,gender
0,0,10,10,True
1,0,9,9,True
2,0,8,8,False
3,0,7,7,True
4,0,6,6,True
5,0,5,5,True
6,0,4,4,True
7,0,3,3,False
8,0,2,2,True
9,0,1,1,True


In [4]:
from holisticai.bias.mitigation.postprocessing.fairsearch.transform import FairSearch

top_n = 10
p = 0.2
alpha = 0.1
fs = FairSearch(top_n=top_n, p=p, alpha=alpha, query_col='query_id', doc_col='doc_id', score_col='score', group_col='gender')

In [5]:
re_rankings  = fs.transform(rankings=rankings)

In [6]:
import pandas as pd
result = pd.concat([rankings[rankings['query_id']==0],re_rankings[re_rankings['query_id']==0]], axis=1)
result

,query_id,doc_id,score,gender,query_id,doc_id,gender,score
0,0,10,10,True,0,10,True,10
1,0,9,9,True,0,9,True,9
2,0,8,8,False,0,8,False,8
3,0,7,7,False,0,7,False,7
4,0,6,6,True,0,6,True,6
5,0,5,5,True,0,5,True,5
6,0,4,4,True,0,4,True,4
7,0,3,3,True,0,3,True,3
8,0,2,2,True,0,2,True,2
9,0,1,1,True,0,1,True,1


In [ ]:
fair = fsc.Fair(k, p, alpha)
mtable = fair.create_unadjusted_mtable()
experimental = fsc.compute_fail_probability(rankings, mtable)